# EEG Classification - Modular Approach

This notebook demonstrates how to use the modular code structure for EEG classification.

## Import Required Modules

In [ ]:
import sys
import os
import numpy as np
import matplotlib.pyplot as plt

# Add the parent directory to the path to import our modules
sys.path.append('..')

# Import our custom modules
from src.data_processing import load_eeg_data, create_binary_labels, create_tri_labels
from src.preprocessing import preprocess_eeg_data, plot_eeg_signal
from src.models import prepare_data_for_model, create_lstm_model, create_bilstm_model, create_lstm_cnn_model, train_model
from src.evaluation import evaluate_binary_model, evaluate_multiclass_model, plot_training_history, print_model_evaluation
from src.visualization import plot_eeg_by_class, visualize_data_clusters

## Load and Prepare Data

In [ ]:
# Set the path to your data directory
data_folder = '../data'

# Load data
all_data, all_names = load_eeg_data(data_folder)
print(f"Loaded {len(all_data)} samples")
print(f"Data shape: {all_data.shape}")

In [ ]:
# Create labels
binary_labels = create_binary_labels(all_names)
tri_labels = create_tri_labels(all_names)

# Display label distribution
binary_counts = np.bincount(binary_labels)
tri_counts = np.bincount(tri_labels)

print("Binary label distribution:")
print(f"Healthy: {binary_counts[0]}")
print(f"Seizure: {binary_counts[1]}")
print("\nThree-class label distribution:")
print(f"Healthy: {tri_counts[0]}")
print(f"Mild Seizure: {tri_counts[1]}")
print(f"Seizure: {tri_counts[2]}")

## Preprocess the Data

In [ ]:
# Plot a raw EEG signal
plot_eeg_signal(all_data[0], title="Raw EEG Signal", stage="Before Preprocessing")

In [ ]:
# Preprocess each EEG signal
processed_data = np.array([preprocess_eeg_data(data) for data in all_data])

# Plot the preprocessed signal
plot_eeg_signal(processed_data[0], title="Preprocessed EEG Signal", stage="After Preprocessing")

## Visualize Data by Class

In [ ]:
# Plot examples from each class (binary classification)
plot_eeg_by_class(processed_data, binary_labels, class_names=['Healthy', 'Seizure'], samples_per_class=2)

In [ ]:
# Plot examples from each class (multi-class classification)
plot_eeg_by_class(processed_data, tri_labels, class_names=['Healthy', 'Mild Seizure', 'Seizure'], samples_per_class=2)

## Visualize Data Clusters

In [ ]:
# Visualize binary class data using PCA
visualize_data_clusters(processed_data, binary_labels, method='pca', class_names=['Healthy', 'Seizure'])

In [ ]:
# Visualize multi-class data using t-SNE (this may take a while)
visualize_data_clusters(processed_data, tri_labels, method='tsne', class_names=['Healthy', 'Mild Seizure', 'Seizure'])

## Prepare Data for Modeling

In [ ]:
# For binary classification
X_train_bin, X_test_bin, y_train_bin, y_test_bin = prepare_data_for_model(processed_data, binary_labels)

# For multi-class classification
X_train_tri, X_test_tri, y_train_tri, y_test_tri = prepare_data_for_model(processed_data, tri_labels)

## Binary Classification with LSTM Model

In [ ]:
# Create the LSTM model for binary classification
input_shape = (X_train_bin.shape[1], 1)  # (timesteps, features)
lstm_model = create_lstm_model(input_shape, num_classes=1)
lstm_model.summary()

In [ ]:
# Train the model
lstm_model, lstm_history = train_model(lstm_model, X_train_bin, y_train_bin, X_test_bin, y_test_bin, epochs=10, batch_size=32)

In [ ]:
# Plot training history
plot_training_history(lstm_history)

In [ ]:
# Evaluate the model
accuracy, y_pred, report, conf_matrix = evaluate_binary_model(lstm_model, X_test_bin, y_test_bin)
print_model_evaluation(accuracy, report, conf_matrix, class_names=['Healthy', 'Seizure'])

## Binary Classification with BiLSTM Model

In [ ]:
# Create the BiLSTM model for binary classification
bilstm_model = create_bilstm_model(input_shape, num_classes=1)
bilstm_model.summary()

In [ ]:
# Train the model
bilstm_model, bilstm_history = train_model(bilstm_model, X_train_bin, y_train_bin, X_test_bin, y_test_bin, epochs=10, batch_size=32)

In [ ]:
# Plot training history
plot_training_history(bilstm_history)

In [ ]:
# Evaluate the model
accuracy, y_pred, report, conf_matrix = evaluate_binary_model(bilstm_model, X_test_bin, y_test_bin)
print_model_evaluation(accuracy, report, conf_matrix, class_names=['Healthy', 'Seizure'])

## Binary Classification with CNN-LSTM Model

In [ ]:
# Create the CNN-LSTM model for binary classification
cnn_lstm_model = create_lstm_cnn_model(input_shape, num_classes=1)
cnn_lstm_model.summary()

In [ ]:
# Train the model
cnn_lstm_model, cnn_lstm_history = train_model(cnn_lstm_model, X_train_bin, y_train_bin, X_test_bin, y_test_bin, epochs=10, batch_size=32)

In [ ]:
# Plot training history
plot_training_history(cnn_lstm_history)

In [ ]:
# Evaluate the model
accuracy, y_pred, report, conf_matrix = evaluate_binary_model(cnn_lstm_model, X_test_bin, y_test_bin)
print_model_evaluation(accuracy, report, conf_matrix, class_names=['Healthy', 'Seizure'])

## Multi-class Classification with CNN-LSTM Model

In [ ]:
# Create the CNN-LSTM model for multi-class classification
input_shape = (X_train_tri.shape[1], 1)  # (timesteps, features)
tri_model = create_lstm_cnn_model(input_shape, num_classes=3)
tri_model.summary()

In [ ]:
# Train the model
tri_model, tri_history = train_model(tri_model, X_train_tri, y_train_tri, X_test_tri, y_test_tri, epochs=10, batch_size=32)

In [ ]:
# Plot training history
plot_training_history(tri_history)

In [ ]:
# Evaluate the model
accuracy, y_pred, report, conf_matrix = evaluate_multiclass_model(tri_model, X_test_tri, y_test_tri)
print_model_evaluation(accuracy, report, conf_matrix, class_names=['Healthy', 'Mild Seizure', 'Seizure'])

## Conclusion

This notebook demonstrates how to use the modular code structure for EEG classification. The modular approach makes the code more maintainable, reusable, and easier to understand. It also simplifies experimenting with different models and configurations.